In [1]:
import pandas as pd

# Load the dataset from the GitHub repository
url = 'https://raw.githubusercontent.com/gregversteeg/LinearCorex/master/tests/data/test_big5.csv'
df = pd.read_csv(url)

# Display basic information about the dataset
print("Number of instances in the dataset:", df.shape[0])
print("Number of columns in the dataset:", df.shape[1])
print("\nFirst 5 rows of the dataset:")
print(df.head())

# Display additional information
print("\nData Types and Non-Null Counts:")
print(df.info())

Number of instances in the dataset: 2000
Number of columns in the dataset: 50

First 5 rows of the dataset:
   blue_q0  red_q1  green_q2  purple_q3  q4  blue_q5  red_q6  green_q7  \
0        2       0         3          1   4        1       4         1   
1        2       0         1          2   2        1       4         3   
2        3       0         2          1   3        1       4         3   
3        2       0         1          1   1        0       4         1   
4        2       0         1          1   3        0       4         3   

   purple_q8  q9  ...  blue_q40  red_q41  green_q42  purple_q43  q44  \
0          2   2  ...         3        3          3           2    3   
1          3   1  ...         2        3          2           2    3   
2          3   0  ...         4        4          2           1    4   
3          3   1  ...         1        2          2           1    3   
4          2   0  ...         3        4          1           3    4   

   blue_q45  r

In [2]:
df = df / 4.0
# print(df.head())

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target (Y) if necessary.
# In the case of autoencoder-like models, we do not have target Y, so we'll treat the whole dataset as X.
X = df.values  # Convert the DataFrame into a NumPy array for model input

# Split the dataset into training (80%) and testing sets (20%)
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# Output the shapes to verify
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (1600, 50)
Test set shape: (400, 50)


In [4]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split

# Convert the DataFrame to a NumPy array
data_array = df.to_numpy()

# Convert the data to a PyTorch tensor
data_tensor = torch.tensor(data_array, dtype=torch.float32)

# Create a PyTorch dataset
dataset = TensorDataset(data_tensor)

# Split the dataset into training and validation sets (80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders for the training and validation sets
batch_size = 32  # You can adjust the batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Display the shape of the tensor to verify
print(f"Data tensor shape: {data_tensor.shape}")
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of validation batches: {len(val_loader)}")

Data tensor shape: torch.Size([2000, 50])
Number of training batches: 50
Number of validation batches: 13


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_dim, output_dim, embedding_dim, hidden_dims=[]):
        super(Encoder, self).__init__()
        # Define the layers of the MLP for shared processing
        dims = [input_dim] + hidden_dims
        self.shared_layers = nn.ModuleList()
        for i in range(len(dims) - 1):
            self.shared_layers.append(nn.Sequential(
                nn.Linear(dims[i], dims[i + 1]),
                nn.ReLU()
            ))

        # Output layers for mu and log_var
        self.fc_mu = nn.Linear(dims[-1], output_dim)
        self.fc_log_var = nn.Linear(dims[-1], output_dim)

        # Learnable embedding vectors e_i for each z_i
        self.e = nn.Parameter(torch.randn(output_dim, embedding_dim))
        self.embedding_dim = embedding_dim
        self.output_dim = output_dim

    def forward(self, x):
        # Pass the input through the shared MLP layers
        for layer in self.shared_layers:
            x = layer(x)  # Shape: (batch_size, hidden_dim)

        # Compute mu and log_var with activation functions
        mu = self.fc_mu(x)             # Shape: (batch_size, output_dim)
        mu = torch.tanh(mu)            # Constrain mu between -1 and 1

        log_var = self.fc_log_var(x)   # Shape: (batch_size, output_dim)
        log_var = torch.sigmoid(log_var)  # Constrain log_var between 0 and 1

        # Reparameterization trick
        std = torch.exp(0.5 * log_var)   # std = exp(0.5 * log_var)
        eps = torch.randn_like(std)
        z = mu + std * eps              # Shape: (batch_size, output_dim)

        # Convert z to \hat{Z} by multiplying each scalar z_i with its own embedding vector e_i
        batch_size = z.size(0)
        z_expanded = z.unsqueeze(2)                       # Shape: (batch_size, output_dim, 1)
        e_expanded = self.e.unsqueeze(0)                  # Shape: (1, output_dim, embedding_dim)
        hat_Z = z_expanded * e_expanded                   # Shape: (batch_size, output_dim, embedding_dim)

        return hat_Z, mu, log_var  # Return mu and log_var for KL divergence computation

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Decoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dims=[]):
        super(Decoder, self).__init__()
        self.input_dim = input_dim      # Number of observed variables (n)
        self.embedding_dim = embedding_dim

        # Learnable query embeddings (e1, e2, ..., en)
        self.query_embeddings = nn.Parameter(torch.randn(input_dim, embedding_dim))

        # MultiheadAttention module with 1 head
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=1, batch_first=True)

        # Layer normalization
        self.layer_norm = nn.LayerNorm(embedding_dim)

        # Define individual MLPs for each observed variable
        dims = [embedding_dim] + hidden_dims + [1]
        self.mlp_layers = nn.ModuleList([
            nn.Sequential(*[
                # Add activation functions
                nn.Sequential(
                    nn.Linear(dims[i], dims[i + 1]),
                    nn.Sigmoid() if i == len(dims) - 2 else nn.ReLU()
                )
                for i in range(len(dims) - 1)
            ])
            for _ in range(input_dim)
        ])

    def forward(self, hat_Z):
        """
        hat_Z: Tensor of shape (batch_size, output_dim, embedding_dim)
        """
        batch_size = hat_Z.size(0)

        # Prepare query embeddings and expand to batch size
        query_embeddings = self.query_embeddings.unsqueeze(0).expand(batch_size, -1, -1)  # Shape: (batch_size, input_dim, embedding_dim)

        # Apply scaled dot-product attention
        attn_output, attn_weights = self.attention(query_embeddings, hat_Z, hat_Z)        # Output shape: (batch_size, input_dim, embedding_dim)

        # Add residual connection and apply layer normalization
        out = self.layer_norm(attn_output + query_embeddings)                             # Shape: (batch_size, input_dim, embedding_dim)

        # Pass each context vector through its corresponding MLP
        x_hat = []
        for i in range(self.input_dim):
            x_i = out[:, i, :]        # Shape: (batch_size, embedding_dim)
            x_i_hat = self.mlp_layers[i](x_i)  # Shape: (batch_size, 1)
            x_hat.append(x_i_hat)
        x_hat = torch.cat(x_hat, dim=1)  # Shape: (batch_size, input_dim)

        return x_hat, attn_weights  # Return attention weights

In [7]:
class Model(nn.Module):
    def __init__(self, input_dim, output_dim, embedding_dim, encoder_hidden_dims=[], decoder_hidden_dims=[]):
        super(Model, self).__init__()
        self.encoder = Encoder(
            input_dim=input_dim,
            output_dim=output_dim,
            embedding_dim=embedding_dim,
            hidden_dims=encoder_hidden_dims
        )
        self.decoder = Decoder(
            input_dim=input_dim,
            embedding_dim=embedding_dim,
            hidden_dims=decoder_hidden_dims
        )

    def forward(self, x):
        hat_Z, mu, log_var = self.encoder(x)  # Obtain \hat{Z}, mu, and log_var from the encoder
        x_hat, attn_weights = self.decoder(hat_Z)  # Reconstruct x from \hat{Z} using the decoder
        return x_hat, attn_weights, mu, log_var  # Return mu and log_var for KL divergence computation

In [8]:
import os
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# Assume that the Encoder, Decoder, and Model classes are already defined

# Define dimensions
input_dim = 50        # Number of observed variables
output_dim = 5        # Output dimension of the encoder (dimension of Z)
embedding_dim = 64    # Embedding dimension for the embeddings e and e_i's
encoder_hidden_dims = [128, 64]  # Hidden dimensions for the encoder
decoder_hidden_dims = [64, 32]   # Hidden dimensions for the decoder

# Instantiate the model
model = Model(
    input_dim=input_dim,
    output_dim=output_dim,
    embedding_dim=embedding_dim,
    encoder_hidden_dims=encoder_hidden_dims,
    decoder_hidden_dims=decoder_hidden_dims
)

# Move the model to the appropriate device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss function and optimizer
criterion = nn.MSELoss(reduction='sum')  # Use sum reduction for reconstruction loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training parameters
num_epochs = 25          # Number of epochs
batch_size = 32          # Batch size (already set in the DataLoader)
print_every = 1          # How often to print loss (in epochs)

# Define the maximum value for the entropy and KL divergence regularization coefficient
max_lambda = 0.5*1e-4  # Adjust this value as needed

# Flag to enable or disable entropy regularizer
use_entropy_regularizer = True  # Set to True to enable, False to disable

# Scheduler function for lambda
def get_lambda(epoch, num_epochs, max_lambda, schedule_type='linear', use_regularizer=True):
    if not use_regularizer:
        return 0.0
    if schedule_type == 'constant':
        # Always return max_lambda
        return max_lambda
    elif schedule_type == 'linear':
        # Linear increase from 0 to max_lambda
        return max_lambda * (epoch / num_epochs)
    elif schedule_type == 'exponential':
        # Exponential increase from 0 to max_lambda
        k = 5  # Adjust this value to control the speed of increase
        return max_lambda * (1 - math.exp(-k * epoch / num_epochs))
    elif schedule_type == 'logarithmic':
        # Logarithmic increase from 0 to max_lambda
        if epoch == 0:
            return 0.0
        else:
            return max_lambda * math.log(epoch + 1) / math.log(num_epochs + 1)
    else:
        raise ValueError(f"Unknown schedule_type: {schedule_type}")

# Initialize a list to store the average attention matrices per epoch
attention_matrices = []

# Training loop with validation
for epoch in range(num_epochs):
    # Compute lambda for the current epoch
    lambda_reg = get_lambda(
        epoch, num_epochs, max_lambda, schedule_type='exponential', use_regularizer=use_entropy_regularizer)

    # Training phase
    model.train()
    running_loss = 0.0          # Accumulates total loss (reconstruction + regularizers)
    running_recon_loss = 0.0    # Accumulates reconstruction loss
    running_kl_loss = 0.0       # Accumulates KL divergence loss
    epoch_attn_weights = []     # List to store attention weights for all batches in the epoch

    for batch_idx, (batch,) in enumerate(train_loader):
        batch = batch.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass: Compute predicted x_hat, attention weights, mu, and log_var
        x_hat, attn_weights, mu, log_var = model(batch)

        # Squeeze attn_weights to get shape: (batch_size, input_dim, output_dim)
        attn_weights = attn_weights.squeeze(1)

        # Collect attention weights for the epoch
        epoch_attn_weights.append(attn_weights.detach().cpu())

        # Compute the reconstruction loss
        recon_loss = criterion(x_hat, batch) / batch.size(0)  # Average over batch

        # Compute KL divergence loss
        kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - torch.exp(log_var)) / batch.size(0)  # Average over batch

        # Initialize entropy_regularizer to zero
        entropy_regularizer = 0.0

        # Compute the entropy regularizer if enabled
        if use_entropy_regularizer:
            # Add a small epsilon to prevent log(0)
            epsilon = 1e-8

            # Compute entropy for each query (input_dim)
            entropy = -torch.sum(attn_weights * torch.log(attn_weights + epsilon), dim=2)  # Shape: (batch_size, input_dim)

            # Sum entropies over queries and average over batch
            entropy_regularizer = torch.mean(torch.sum(entropy, dim=1))  # Scalar

        # Total loss
        loss = recon_loss + kl_loss + lambda_reg * entropy_regularizer

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate training losses
        running_loss += loss.item()
        running_recon_loss += recon_loss.item()
        running_kl_loss += kl_loss.item()

    # Compute average losses for training
    avg_train_loss = running_loss / len(train_loader)
    avg_train_recon_loss = running_recon_loss / len(train_loader)
    avg_train_kl_loss = running_kl_loss / len(train_loader)

    # Compute the average attention matrix for the epoch
    epoch_attn_weights_tensor = torch.cat(epoch_attn_weights, dim=0)  # Shape: (num_samples_in_epoch, input_dim, output_dim)
    avg_attn_weights_epoch = epoch_attn_weights_tensor.mean(dim=0)    # Shape: (input_dim, output_dim)
    avg_attn_weights_epoch_np = avg_attn_weights_epoch.numpy()

    # Transpose to have shape (output_dim, input_dim) so that queries are on x-axis and keys on y-axis
    attention_matrices.append(avg_attn_weights_epoch_np.T)

    # Validation phase
    model.eval()
    val_loss = 0.0          # Accumulates total loss (reconstruction + regularizers)
    val_recon_loss = 0.0    # Accumulates reconstruction loss
    val_kl_loss = 0.0       # Accumulates KL divergence loss
    with torch.no_grad():
        for batch_idx, (batch,) in enumerate(val_loader):
            batch = batch.to(device)

            # Forward pass for validation
            x_hat, attn_weights, mu, log_var = model(batch)

            # Compute the reconstruction loss
            recon_loss = criterion(x_hat, batch) / batch.size(0)  # Average over batch

            # Compute KL divergence loss
            kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - torch.exp(log_var)) / batch.size(0)  # Average over batch

            # Initialize entropy_regularizer to zero
            entropy_regularizer = 0.0

            # Compute the entropy regularizer if enabled
            if use_entropy_regularizer:
                attn_weights = attn_weights.squeeze(1)  # Shape: (batch_size, input_dim, output_dim)

                # Add a small epsilon to prevent log(0)
                epsilon = 1e-8

                # Compute entropy for each query (input_dim)
                entropy = -torch.sum(attn_weights * torch.log(attn_weights + epsilon), dim=2)  # Shape: (batch_size, input_dim)

                # Sum entropies over queries and average over batch
                entropy_regularizer = torch.mean(torch.sum(entropy, dim=1))  # Scalar

            # Total loss
            loss = recon_loss + kl_loss + lambda_reg * entropy_regularizer

            # Accumulate validation losses
            val_loss += loss.item()
            val_recon_loss += recon_loss.item()
            val_kl_loss += kl_loss.item()

    # Compute average losses for validation
    avg_val_loss = val_loss / len(val_loader)
    avg_val_recon_loss = val_recon_loss / len(val_loader)
    avg_val_kl_loss = val_kl_loss / len(val_loader)

    # Print average losses for the epoch
    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Lambda: {lambda_reg:.6f}, '
              f'Train Total Loss: {avg_train_loss:.4f}, '
              f'Train Recon Loss: {avg_train_recon_loss:.4f}, '
              f'Train KL Loss: {avg_train_kl_loss:.4f}, '
              f'Val Total Loss: {avg_val_loss:.4f}, '
              f'Val Recon Loss: {avg_val_recon_loss:.4f}, '
              f'Val KL Loss: {avg_val_kl_loss:.4f}')

Epoch [1/25], Lambda: 0.000000, Train Total Loss: 4.7854, Train Recon Loss: 4.6788, Train KL Loss: 0.1066, Val Total Loss: 4.5429, Val Recon Loss: 4.5408, Val KL Loss: 0.0020
Epoch [2/25], Lambda: 0.000009, Train Total Loss: 4.4557, Train Recon Loss: 4.4535, Train KL Loss: 0.0015, Val Total Loss: 4.5333, Val Recon Loss: 4.5313, Val KL Loss: 0.0013
Epoch [3/25], Lambda: 0.000016, Train Total Loss: 4.4518, Train Recon Loss: 4.4495, Train KL Loss: 0.0011, Val Total Loss: 4.5281, Val Recon Loss: 4.5261, Val KL Loss: 0.0009
Epoch [4/25], Lambda: 0.000023, Train Total Loss: 4.4488, Train Recon Loss: 4.4462, Train KL Loss: 0.0009, Val Total Loss: 4.5354, Val Recon Loss: 4.5330, Val KL Loss: 0.0007
Epoch [5/25], Lambda: 0.000028, Train Total Loss: 4.4520, Train Recon Loss: 4.4492, Train KL Loss: 0.0008, Val Total Loss: 4.5298, Val Recon Loss: 4.5271, Val KL Loss: 0.0007
Epoch [6/25], Lambda: 0.000032, Train Total Loss: 4.4522, Train Recon Loss: 4.4493, Train KL Loss: 0.0006, Val Total Loss: 4.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import seaborn as sns
from sklearn.metrics import adjusted_rand_score

# Assuming 'df' is your original dataframe

# Identify column prefixes for each true factor
factor_columns = {
    'Factor1': [col for col in df.columns if col.startswith('blue')],
    'Factor2': [col for col in df.columns if col.startswith('green')],
    'Factor3': [col for col in df.columns if col.startswith('purple')],
    'Factor4': [col for col in df.columns if col.startswith('red')],
    'Factor5': [col for col in df.columns if col.startswith('q')]
}

# Map factor names to column indices
factor_indices = {}
for factor_name, columns in factor_columns.items():
    indices = [df.columns.get_loc(col) for col in columns]
    factor_indices[factor_name] = indices

# Create a new ordering of indices
new_order = []
for factor_name in factor_columns.keys():
    new_order.extend(factor_indices[factor_name])

# Ensure all indices are included
assert len(new_order) == df.shape[1], "Not all indices are included in the new order."

In [ ]:
# Rearranging attention matrices for the animation
reordered_attention_matrices = []

for attn_matrix in attention_matrices:
    # attn_matrix has shape (output_dim, input_dim)
    # Rearrange the columns (observed variables) according to new_order
    attn_matrix_reordered = attn_matrix[:, new_order]  # Shape: (output_dim, input_dim)
    reordered_attention_matrices.append(attn_matrix_reordered)

# Convert the list to a NumPy array for animation
attention_matrices_array = np.stack(reordered_attention_matrices)  # Shape: (num_epochs, output_dim, input_dim)

# Determine the number of frames (epochs)
num_frames = attention_matrices_array.shape[0]

# Set up the figure and axis for the animation
fig, ax = plt.subplots(figsize=(10, 6))

# Initialize the heatmap with empty data
im = ax.imshow(np.zeros_like(attention_matrices_array[0]), aspect='auto', cmap='viridis', vmin=0, vmax=1)
ax.set_xlabel('Observed Variables')
ax.set_ylabel('Latent Factors')
title = ax.set_title('Attention Matrix at Epoch 1')

# Function to initialize the heatmap
def init():
    data = attention_matrices_array[0]
    im.set_data(data)
    title.set_text('Attention Matrix at Epoch 1')
    return [im, title]

# Function to update the heatmap for each frame
def update(frame):
    data = attention_matrices_array[frame]
    im.set_data(data)
    title.set_text(f'Attention Matrix at Epoch {frame + 1}')
    return [im, title]

# Create the animation
ani = FuncAnimation(fig, update, frames=num_frames, init_func=init, blit=True)

# Close the figure to prevent the static plot from displaying
plt.close(fig)

# Display the animation in the notebook
display(HTML(ani.to_jshtml()))

# Save the animation to an MP4 file (optional)
ani.save('attention_animation.mp4', writer='ffmpeg', fps=2)

In [ ]:
# Compute average attention matrices (assuming you have 'model', 'train_loader', and 'val_loader')
def compute_average_attention(model, dataloader, device):
    model.eval()  # Ensure model is in evaluation mode
    total_attn = None
    num_samples = 0

    with torch.no_grad():
        for data in dataloader:
            inputs = data[0].to(device)

            # Forward pass up to obtaining attention weights
            hat_Z = model.encoder(inputs)
            _, attn_weights, _, _ = model.decoder(hat_Z)

            # attn_weights shape: (batch_size, input_dim, output_dim)
            batch_size = attn_weights.size(0)
            if total_attn is None:
                total_attn = attn_weights.sum(dim=0)  # Sum over batch dimension
            else:
                total_attn += attn_weights.sum(dim=0)
            num_samples += batch_size

    # Average the attention weights
    avg_attn = total_attn / num_samples

    return avg_attn.cpu().numpy()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Compute average attention matrices
avg_attn_train = compute_average_attention(model, train_loader, device)
avg_attn_val = compute_average_attention(model, val_loader, device)

In [ ]:
# Calculating ARI
input_dim = avg_attn_train.shape[0]   # Number of observed variables
output_dim = avg_attn_train.shape[1]  # Number of latent factors

# Step 1: Assign each observed variable to the latent factor with the highest attention weight
predicted_labels = np.argmax(avg_attn_train, axis=1)  # Shape: (input_dim,)

# Step 2: Create true labels based on ideal groups
true_labels = np.full(input_dim, -1)  # Initialize with -1

# Map factor names to indices
factor_names = ['Factor1', 'Factor2', 'Factor3', 'Factor4', 'Factor5']
factor_name_to_index = {name: idx for idx, name in enumerate(factor_names)}

# Assign labels to observed variables based on factors
for factor_name, indices in factor_indices.items():
    factor_idx = factor_name_to_index[factor_name]
    true_labels[indices] = factor_idx

# Ensure all observed variables have been assigned
assert np.all(true_labels >= 0), "Some observed variables have not been assigned a true label"

# Step 3: Compute Adjusted Rand Index
ari = adjusted_rand_score(true_labels, predicted_labels)

print("Adjusted Rand Index:", ari)

In [ ]:
# Transpose the average attention matrices
avg_attn_train_transposed = avg_attn_train.T  # Shape: (output_dim, input_dim)
avg_attn_val_transposed = avg_attn_val.T      # Shape: (output_dim, input_dim)

# Rearrange the columns (observed variables) using new_order
avg_attn_train_reordered = avg_attn_train_transposed[:, new_order]
avg_attn_val_reordered = avg_attn_val_transposed[:, new_order]

def plot_attention_heatmap(attn_matrix, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        attn_matrix,
        cmap='viridis',
        vmin=0,
        vmax=1,
        cbar_kws={'label': 'Attention Weight'}
    )
    plt.title(title)
    plt.xlabel('Observed Variables')
    plt.ylabel('Latent Factors')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

# Plot average attention matrices with fixed color scaling
plot_attention_heatmap(avg_attn_train_reordered, 'Average Attention Matrix - Training Set')